In [4]:
import warnings
warnings.filterwarnings("ignore")

In [5]:
import numpy as np

from IPython.display import clear_output

In [9]:
!pip install ipython jupyter

In [14]:
!pip install tensorflow-gpu
!pip install h5py

clear_output()

In [15]:
#download dataset
!wget https://data.mendeley.com/datasets/rscbjbr9sj/2/files/41d542e7-7f91-47f6-9ff2-dd8e5a5a7861/ChestXRay2017.zip

clear_output()

In [16]:
!ls

ChestXRay2017.zip
week-04-pneumonia-image-classification.ipynb


In [17]:
#Inspecting contents of ChestXRay2017.zip
from zipfile import ZipFile

with ZipFile("./ChestXRay2017.zip", "r") as f:
    f.extractall(path = "./")

In [20]:
!ls

ChestXRay2017.zip
__MACOSX
chest_xray
week-04-pneumonia-image-classification.ipynb


In [19]:
!ls chest_xray

test  train


In [21]:
!ls chest_xray/train


NORMAL    PNEUMONIA


In [22]:
!ls chest_xray/test


NORMAL    PNEUMONIA


In [23]:
#Creating ImageGenerator instances for training, validating and testing
from pathlib import Path

train_files = "./chest_xray/train/"
test_files = "./chest_xray/test/"

positive_class_folder_name = "PNEUMONIA"
negative_class_folder_name = "NORMAL"

In [24]:
# training files

# number of X-RAYS which show Pneumonia
P = Path(train_files + positive_class_folder_name + "/")
temp = list(P.glob("*.jpeg"))
print("Training-Pneumonia - %i " % len(temp))

# number of X-RAYS which are normal
P = P = Path(train_files + negative_class_folder_name + "/")
temp = list(P.glob("*.jpeg"))
print("Training-Normal - %i " % len(temp))

Training-Pneumonia - 3883 
Training-Normal - 1349 


In [25]:
# testing files

# number of X-RAYS which show Pneumonia
P = Path(test_files + positive_class_folder_name + "/")
temp = list(P.glob("*.jpeg"))
print("Testing-Pneumonia - %i " % len(temp))

# number of X-RAYS which are normal
P = P = Path(test_files + negative_class_folder_name + "/")
temp = list(P.glob("*.jpeg"))
print("Testing-Normal - %i " % len(temp))

Testing-Pneumonia - 390 
Testing-Normal - 234 


In [26]:
# creating ImageDataGenerators for training

"""
The training data generator must have other augmentation parameters in addition to the rescaling
parameter `rescale`.

Why?

-   The model needs to trained to learn to focus on the essential pieces that determine the answer
    inspite of the possible variations it may find in new X-RAY images. Augmentation parameters like
    random zooming and flipping simulate unpredictable variations
-   The test images should be just like the real-life examples of X-RAY images: they don't have the simulated
    effects like those in the training data.
-   The images used for validation during training must also be like the test-images.
    
"""


from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_generator = ImageDataGenerator(featurewise_center = True,
                                     shear_range = 0.2,
                                     rescale = 1.0/255,
                                     horizontal_flip = True,
                                     validation_split = 0.2)

test_generator = ImageDataGenerator(rescale = 1.0/255)

In [28]:
# create batch iterators for the train, validation and test data

train_iterator = train_generator.flow_from_directory(train_files, 
                                                     target_size = (150, 150),
                                                     classes = [positive_class_folder_name, negative_class_folder_name],
                                                     batch_size = 256, 
                                                     shuffle = True, 
                                                     class_mode = "binary",
                                                     subset = "training")
                                                    
validation_iterator = train_generator.flow_from_directory(train_files, 
                                                          target_size = (150, 150),
                                                          classes = [positive_class_folder_name, negative_class_folder_name],
                                                          batch_size = 128, 
                                                          shuffle = False, 
                                                          class_mode = "binary",
                                                          subset = "validation")

test_iterator = test_generator.flow_from_directory(test_files, 
                                                   target_size = (150, 150), 
                                                   classes = [positive_class_folder_name, negative_class_folder_name],
                                                   batch_size = 64, 
                                                   shuffle = False,
                                                   class_mode = "binary")

Found 4187 images belonging to 2 classes.
Found 1045 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [29]:
# count for class instances in training data

class1 = sum(train_iterator.classes)
class0 = len(train_iterator.classes) - class1

print("Label 0 - %i, Label 1 - %i" % (class0, class1))

Label 0 - 3107, Label 1 - 1080


In [30]:
# count for class instances in validation data

class1 = sum(validation_iterator.classes)
class0 = len(validation_iterator.classes) - class1

print("Label 0 - %i, Label 1 - %i" % (class0, class1))

Label 0 - 776, Label 1 - 269


In [31]:
# count for class instances in testing data

class1 = sum(test_iterator.classes)
class0 = len(test_iterator.classes) - class1

print("Label 0 - %i, Label 1 - %i" % (class0, class1))

Label 0 - 390, Label 1 - 234


In [32]:
#As per as the output of the previous 3 cells, we can deduce that ImageGenerator assigns the numeric labels as:
#negative_class_folder_name -> the label 1
#positive_class_folder_name -> the label 0
# defining category label weights as per imbalance amount of the training dataset

from sklearn.utils.class_weight import compute_class_weight

y = train_iterator.classes
labels = np.unique(y)

train_class_weights = compute_class_weight("balanced", labels, y)
train_class_weights

array([0.67380109, 1.93842593])

In [33]:
# Transfer Learning using the InceptionV3 pretrained CNN
# are the Colab GPUs available to tensorflow?

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 5681680000031690511
]


In [34]:
from tensorflow.keras.backend import clear_session

from tensorflow.keras.layers import Dropout, GlobalAveragePooling2D, BatchNormalization, Dense

from tensorflow.keras.models import Sequential, load_model, save_model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [35]:
#Modify the Inception V3 CNN as per as the output requirements
clear_session()

base_model = InceptionV3(weights = "imagenet", include_top = False, input_shape = (150, 150, 3))
clear_output()

base_model.trainable = False
myModel = Sequential([base_model,
                      GlobalAveragePooling2D(),
                      Dense(1024, activation = "relu"),
                      Dropout(0.3),
                      Dense(512, activation = "relu"),
                      Dropout(0.3),
                      Dense(256, activation = "relu"),
                      Dropout(0.3),
                      Dense(128, activation = "relu"),
                      Dropout(0.3),
                      Dense(1, activation = "sigmoid")])

myModel.compile(RMSprop(), loss = "binary_crossentropy", metrics = ["accuracy"])
myModel.summary()

W0928 21:17:07.391397 4619916736 deprecation.py:323] From /Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_v3 (Model)         (None, 3, 3, 2048)        21802784  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               1

In [40]:
#Train the model
history = myModel.fit_generator(train_iterator, 
                                steps_per_epoch = len(train_iterator),
                                epochs = 10,
                                verbose = 1,
                                workers = 20,
                                validation_data = validation_iterator, 
                                validation_steps = len(validation_iterator),
                                class_weight = train_class_weights)

Epoch 1/10
17/17 [==============================] - 314s 18s/step - loss: 0.2112 - acc: 0.9157 - val_loss: 1.0538 - val_acc: 0.7675
Epoch 2/10
17/17 [==============================] - 393s 23s/step - loss: 0.2265 - acc: 0.9126 - val_loss: 0.4659 - val_acc: 0.8498
Epoch 3/10
17/17 [==============================] - 320s 19s/step - loss: 0.2010 - acc: 0.9236 - val_loss: 1.3676 - val_acc: 0.5818
Epoch 4/10
17/17 [==============================] - 320s 19s/step - loss: 0.2197 - acc: 0.9123 - val_loss: 1.0014 - val_acc: 0.7694
Epoch 5/10
17/17 [==============================] - 370s 22s/step - loss: 0.1984 - acc: 0.9200 - val_loss: 0.3314 - val_acc: 0.8574
Epoch 6/10
17/17 [==============================] - 400s 24s/step - loss: 0.2065 - acc: 0.9159 - val_loss: 0.2914 - val_acc: 0.8632
Epoch 7/10
17/17 [==============================] - 293s 17s/step - loss: 0.1873 - acc: 0.9219 - val_loss: 0.3177 - val_acc: 0.8727
Epoch 8/10
17/17 [==============================] - 315s 19s/step - loss: 0.

In [42]:
#Save the model
save_model(myModel, "./myModel_after_10_epochs", overwrite = True, include_optimizer = True)


In [44]:
# Load myModel from local directory if required
myModel = load_model("./myModel_after_10_epochs")


In [46]:
# Compute Evalution Metrics
from sklearn.metrics import classification_report


yhat = myModel.predict_generator(test_iterator,
                                 steps = len(test_iterator),
                                 verbose = 0)  
mask = yhat > 0.5
yhat[mask] = 1
yhat[~mask] = 0

y = test_iterator.classes

target_names = [positive_class_folder_name, negative_class_folder_name]
print(classification_report(y, yhat, target_names = target_names))

              precision    recall  f1-score   support

   PNEUMONIA       0.64      1.00      0.78       390
      NORMAL       0.92      0.05      0.09       234

    accuracy                           0.64       624
   macro avg       0.78      0.52      0.43       624
weighted avg       0.74      0.64      0.52       624



In [47]:
#Compute Evaluation Metrics
from sklearn.metrics import classification_report


yhat = myModel.predict_generator(test_iterator,
                                 steps = len(test_iterator),
                                 verbose = 0)  
mask = yhat > 0.5
yhat[mask] = 1
yhat[~mask] = 0

y = test_iterator.classes

target_names = [positive_class_folder_name, negative_class_folder_name]
print(classification_report(y, yhat, target_names = target_names))


              precision    recall  f1-score   support

   PNEUMONIA       0.64      1.00      0.78       390
      NORMAL       0.92      0.05      0.09       234

    accuracy                           0.64       624
   macro avg       0.78      0.52      0.43       624
weighted avg       0.74      0.64      0.52       624



In [54]:
#test output
!pip3 install matplotlib
!pip3 install cv2
from random import randint
import matplotlib.pyplot as plt
#import cv2

from IPython import get_ipython
%matplotlib inline

label_dict = {0:"PNEUMONIA", 1:"NORMAL"}

test_file_names = test_iterator.filenames
n = len(test_file_names)

plt.rcParams["figure.figsize"] = 12, 10

for i in range(5):
    index = randint(0, n - 1)
    file_name = test_files + test_file_names[index]
    
    image = cv2.imread(file_name, cv2.IMREAD_COLOR)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    ground_truth_label = label_dict[int(y[index])]
    predicted_label = label_dict[int(yhat[index])]
    
    plt.subplot(5, 1, i + 1)
    plt.axis("off")
    plt.imshow(image)
    
    title = "Ground Truth = " + ground_truth_label + ", Predicted Label = " + predicted_label
    plt.title(title)

plt.tight_layout()
plt.show()

  ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


NameError: name 'cv2' is not defined